## *2.9* 当列表不是首选时

虽然列表灵活简单，但不能很好的面对各类需求，例如要存放1000万个浮点数的话，数组(array)的效率更高，如果需要频繁对序列做先进先出的操作，deque的速度应该更快

### 2.9.1 数组

如果只需要一个包含数字的列表, 那么array.array比list更高效, 数组支持所有跟可变序列有关的操作，包括.pop, .insert和.extend。 另外数组还提供从文件中获取读取和存入文件的.fromfile和.tofile操作

Python创建数组需要一个类型码，这个类型码用来表示在底层的C语言应该存放怎样的数据类型，python不会允许在数组中存放指定类型之外的数据

In [2]:
from array import array  # 引入array类型
from random import random

floats = array('d', (random() for i in range(10**7)))  # 利用一个可迭代对象来建立一个双精度浮点数组(类型码是'd')
floats[-1]  # 查看数组的最后一个元素


0.14994204153718582

In [2]:
fp = open('floats.bin', 'wb')
floats.tofile(fp)  # 把数组存入一个二进制文件里
fp.close()

In [3]:
floats2 = array('d')  # 新建一个空的双进度浮点数组
fp = open('floats.bin', 'rb')  # 将1000万浮点数从文件中读取出来
floats2.fromfile(fp, 10**7)  # 写入到空的数组中
fp.close()

floats == floats2

True

另一个快速序列化数字类型的方法是使用pickle模块。pickle.dump处理浮点数组的速度几乎跟array.tofile一样快，前者可以处理几乎所有的内置数字类型。

In [ ]:
# 从python3.4开始，数组类型不支持诸如list.sort()这种就地排序方法。要给数组排序需要sorted函数新建一个数组
a = array(a.typecode, sorted(a))

### 2.9.2 内存视图

memoryview是一个内置类，它能让用户在不经过复制的情况下操作同一数组的不同切片，在数据结构之前共享内存，这个功能在处理大型数据集合的时候非常重要

memoryview.cast的概念跟数组模块相类似，能用不同的方式读写同一块内存数据，而且内存字节不会随意移动。memoryview.cast会把同一块内存里的内容打包成一个全新的memoryview对象给你

In [3]:
numbers = array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)  # 利用含有5个短整型有符号整数的数组创建一个内存视图
memv_oct = memv.cast('B')  # 创建一个memv_oct, 这一次是把memv里的内容转换成'B'类型,也就是无符号字符
memv_oct.tolist()  # 以列表的形式查看memv_oct的内容

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [5]:
memv_oct[5] = 1  # 把位置5的字节赋值为4
numbers  # 因为把两个字节的整数的高位字节改成了4, 所以这个有符号整数的值就变成了1024

array('h', [-2, -1, 256, 1, 2])

### 2.9.3 NumPy和SciPy

NumPy实现了多维同质数组和矩阵,SciPy是基于NumPy的另一个库,它提供了很多跟科学计算有关的算法

### 2.9.4 双向队列和其他形式的队列

利用.append和.pop方法,我们可以把列表当做栈或者队列来用,但是操作列表的第一个元素是非常耗时的,因为这会牵扯到移动列表里的所有元素

collections.deque类(双向队列)是一个线程安全,可以快速从两端添加或删除元素的数据类型.如果想要有一种数据类型来存放"最近使用的几个元素",deque也是一个很好的选择.这是因为在新建一个双向队列的时候,可以指定这个队列的大小,如果这个队列满了,可以反向删除过期的元素,然后在尾端添加新的元素.

In [6]:
from collections import deque

dq = deque(range(10), maxlen=10)  # maxlen 是一个可选参数，代表这个队列可容纳的元素数量,一旦设定则不能更改
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [8]:
dq.rotate(3)  # 队列的旋转操作接受一个参数 n,当 n>0 时队列最右边的n个元素会被移动到队列的左边.当 n<0 时,最左边的n个元素会被移动到右边
dq

deque([4, 5, 6, 7, 8, 9, 0, 1, 2, 3])

In [9]:
dq.rotate(-4)
dq

deque([8, 9, 0, 1, 2, 3, 4, 5, 6, 7])

In [11]:
dq.extendleft([10, 20, 30, 40])  # extendleft(iter)方法会把迭代器里的元素逐个添加到双向队列的左边，因此迭代器的元素会逆向出现在队列里
dq

deque([40, 30, 20, 10, 40, 30, 20, 10, 8, 9])

双向队列实现了大部分列表的所拥有的方法，也有一些额外的符合自身设计的方法，比如说popleft和rotate。但从队列中间操作元素会慢一些，因为它只对头尾操作进行了优化

append和popleft都是原子操作，也就是说deque可以在多线程中安全地充当先进先出的栈使用，而使用者不需要担心资源锁的问题。

除了deque之外，还有些其他的python标准库也有队列的实现:
**queue**
    提供了同步(线程安全类)Queue,LifoQueue和PriorityQueue, 不同线程可以利用这些数据类型来交换信息.这三个类的构造方法都存在一个可选参数maxsize,它接收正整数作为参数值,用来限定队列大小.如果队列满了,这些类不会删除旧元素来腾出位置,而是将其锁住,待到其他线程移除了某个元素腾出位置.这一特性很适合用来控制线程活跃的数量。
**asyncio**
    里面有 Queue、LifoQueue、PriorityQueue 和JoinableQueue，这些类受到 queue 和 multiprocessing 模块的影响，但是为异步编程里的任务管理提供了专门的便利。
**heapq**
    heapq没有队列类,而是提供了heappush和heappop方法,让用户可以把可变序列当作堆队列或者优先队列来使用。